In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
df = pd.read_excel(r"D:\Pharma_Inventory_Optimization_Project\03_Model\Safety_Stock_Model.xlsx")

In [36]:
df.head()

,Item,Average_Demand,StdDev_Demand,Lead_Time
0,1,21.981599,8.468922,7
1,2,58.574151,20.093015,7
2,3,36.642223,13.179441,7
3,4,22.010241,8.403898,7
4,5,18.358708,7.265167,7


In [37]:
df.columns

Index(['Item', 'Average_Demand', 'StdDev_Demand', 'Lead_Time'], dtype='str')

In [38]:
df["Service_Level"] = 0.95
df["Z_value"] = 1.65

In [39]:
df.head()

,Item,Average_Demand,StdDev_Demand,Lead_Time,Service_Level,Z_value
0,1,21.981599,8.468922,7,0.95,1.65
1,2,58.574151,20.093015,7,0.95,1.65
2,3,36.642223,13.179441,7,0.95,1.65
3,4,22.010241,8.403898,7,0.95,1.65
4,5,18.358708,7.265167,7,0.95,1.65


In [40]:
df.rename(columns={"StdDev_Demand": "Std_Demand"}, inplace=True)
df.rename(columns={"Average_Demand": "Avg_Demand"}, inplace=True)

In [41]:
df["Avg_Demand"] = round(df["Avg_Demand"],2)
df["Std_Demand"] = round(df["Std_Demand"],2)

In [42]:
df.head()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value
0,1,21.98,8.47,7,0.95,1.65
1,2,58.57,20.09,7,0.95,1.65
2,3,36.64,13.18,7,0.95,1.65
3,4,22.01,8.40,7,0.95,1.65
4,5,18.36,7.27,7,0.95,1.65


In [43]:
df["Safety_Stock"] = round(df["Std_Demand"] * df["Z_value"] * np.sqrt(df["Lead_Time"]),0)
#square root of lead time
# I did round of till 0 cause the safety stock value can never be in points

In [45]:
df["Reorder_Point"] = round((df["Avg_Demand"] * df["Lead_Time"]) + df["Safety_Stock"],0)

In [49]:
df["CV_%"] = round((df["Std_Demand"] / df["Avg_Demand"]) * 100,0)

In [50]:
df.tail()

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%
45,46,58.64,20.22,3,0.95,1.65,58.0,234.0,34.0
46,47,22.00,8.42,3,0.95,1.65,24.0,90.0,38.0
47,48,51.35,17.88,3,0.95,1.65,51.0,205.0,35.0
48,49,29.34,10.87,3,0.95,1.65,31.0,119.0,37.0
49,50,65.88,22.42,3,0.95,1.65,64.0,262.0,34.0


In [55]:
df.nlargest(1,"CV_%")

,Item,Avg_Demand,Std_Demand,Lead_Time,Service_Level,Z_value,Safety_Stock,Reorder_Point,CV_%
4,5,18.36,7.27,7,0.95,1.65,32.0,161.0,40.0


In [58]:
df["CV_%"].min()

np.float64(34.0)